<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/train_rf_xgb_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version  # Check if CUDA is installed
!nvidia-smi  # Check if GPU is available


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Mon Feb 24 20:21:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0       

In [ ]:
# ✅ Step 2: Set Environment Paths for CUDA 11.8
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.8'
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-11.8/lib64'


In [ ]:
#!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 216.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 224.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 130.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.2/851.2 kB 197.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 179.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 192.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import numba
import cudf
import cuml
import tensorflow as tf

print("✅ NumPy Version:", np.__version__)
print("✅ Numba Version:", numba.__version__)
print("✅ TensorFlow Version:", tf.__version__)
print("✅ Available GPUs:", tf.config.list_physical_devices('GPU'))


✅ NumPy Version: 1.26.4
✅ Numba Version: 0.60.0
✅ TensorFlow Version: 2.18.0
✅ Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import tensorflow as tf

print("TF Version:", tf.__version__)
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# Run a test to ensure GPU acceleration is working
tf.debugging.set_log_device_placement(True)

# Test a simple TensorFlow computation on GPU
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0]])
    b = tf.constant([[4.0, 5.0, 6.0]])
    result = tf.matmul(a, tf.transpose(b))
    print("Matrix multiplication result:\n", result)


TF Version: 2.18.0
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Matrix multiplication result:
 tf.Tensor([[32.]], shape=(1, 1), dtype=float32)


In [ ]:
#!pip install gym-anytrading


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
#!pip install stable-baselines3[extra]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
import os
import time
import gc
import numpy as np
import pandas as pd
import cudf
import cuml
import xgboost as xgb
import yfinance as yf
import gymnasium as gym  # ✅ Use gymnasium instead of gym
import gym_anytrading
from gymnasium.envs.registration import registry, register
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from google.colab import drive


In [ ]:
# ✅ Step 5: Detect GPU Properly
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 Using GPU for Accelerated Training!")
except (ImportError, Exception) as e:
    GPU_AVAILABLE = False
    print(f"⚠️ GPU not available or cuDF not installed: {e}")
    print("🔄 Switching to CPU mode.")

# ✅ Step 6: Download Stock Data
def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker} stock data...")
            df_live = yf.download(ticker, period=period, interval=interval)
            if not df_live.empty:
                print("✅ Successfully downloaded stock data!")
                df_live.reset_index(inplace=True)
                return df_live
            raise ValueError("Downloaded data is empty. Retrying...")
        except Exception as e:
            print(f"⚠️ Error: {e}. Retrying in {attempt * 5} seconds...")
            time.sleep(attempt * 5)
    print("❌ Failed to download stock data after multiple attempts.")
    return None

df_live = download_stock_data("TSLA")
if df_live is None:
    print("⚠️ Using previously saved dataset instead.")
    file_path = '/content/drive/My Drive/teslafeature_engineered_dataset.csv'
    df_live = pd.read_csv(file_path)

🚀 Using GPU for Accelerated Training!
Attempt 1: Downloading TSLA stock data...


[*********************100%***********************]  1 of 1 completed

✅ Successfully downloaded stock data!


In [ ]:
# ✅ Choose Dataset for Training
df = df_live.copy()

In [ ]:
# ✅ Step 3: Fix Potential Multi-Index Issues (Optional Safety Check)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

In [ ]:
# ✅ Step 2: Feature Engineering
# =============================
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['STD_20'] = df['Close'].rolling(window=20).std()
df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
df['Lowest_Low'] = df['Low'].rolling(window=14).min()
df['Highest_High'] = df['High'].rolling(window=14).max()
df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100
df.dropna(inplace=True)

In [ ]:
# ✅ Create Trade Labels
df['Future_Close'] = df['Close'].shift(-10)
df['Price_Change'] = (df['Future_Close'] - df['Close']) / df['Close']
df['Target'] = np.where(df['Price_Change'] > 0.03, 1, 0)
df.dropna(inplace=True)

In [ ]:
# ✅ Check if GPU is truly available (not just detected)
GPU_AVAILABLE = False  # Default to CPU

try:
    import cupy as cp
    import cudf
    import cuml

    # ✅ Test if cuDF and GPU support work properly (catches CUDA driver issues)
    _ = cudf.DataFrame({'test': [1, 2, 3]})

    # ✅ Test TensorFlow GPU availability
    import tensorflow as tf
    if tf.config.list_physical_devices('GPU'):
        print("🚀 TensorFlow detected GPU for training!")
    else:
        print("⚠️ TensorFlow does not detect GPU. Running on CPU.")

    GPU_AVAILABLE = True
    print("🚀 Using GPU with cuDF & TensorFlow!")

except Exception as e:
    print(f"⚠️ GPU detected but not usable: {e}")
    print("🔄 Switching to CPU mode.")
    GPU_AVAILABLE = False

# ✅ Define Features & Target
features = ['SMA_20', 'STD_20', 'Upper_Band', 'Lower_Band', 'Stoch']
X = df[features]
y = df['Target']

# ✅ Convert DataFrame to GPU (if available)
if GPU_AVAILABLE:
    try:
        X = cudf.DataFrame.from_pandas(X)  # Convert features to cuDF
        y = cudf.Series(y.values)  # Convert target to cuDF
    except Exception as e:
        print(f"⚠️ GPU Conversion Failed: {e}. Using CPU Instead.")
        X = X  # Stay in Pandas format
        y = y
        GPU_AVAILABLE = False  # Fallback to CPU

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# ✅ Ensure cuDF/Pandas data is correctly formatted before model training
print("✅ Data Type:", type(X_train))

# ✅ Verify TensorFlow GPU Availability
import tensorflow as tf
print("TF Version:", tf.__version__)
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# ✅ Verify CUDA & CuDNN versions
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvcc --version
!nvidia-smi


🚀 TensorFlow detected GPU for training!
🚀 Using GPU with cuDF & TensorFlow!
✅ Data Type: <class 'cudf.core.dataframe.DataFrame'>
TF Version: 2.18.0
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


Mon Feb 24 20:22:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             27W /   70W |     150MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# ✅ Train Random Forest
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)
rf_accuracy = accuracy_score(y_test, rf_model.predict(X_test))
print(f"✅ Random Forest Accuracy: {rf_accuracy:.4f}")

del X_train, X_test, y_train, y_test  # Free memory
gc.collect()

/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:363: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return init_func(self, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


✅ Random Forest Accuracy: 0.6992


831

In [ ]:
# ✅ Train Gradient Boosting Model
X_train_pandas = X.to_pandas() if GPU_AVAILABLE else X
y_train_pandas = y.to_pandas() if GPU_AVAILABLE else y

gb_model = xgb.XGBClassifier(n_estimators=50, learning_rate=0.1, tree_method='hist' if GPU_AVAILABLE else 'exact', random_state=42)
gb_model.fit(X_train_pandas, y_train_pandas)
gb_accuracy = accuracy_score(y_train_pandas, gb_model.predict(X_train_pandas))
print(f"✅ Gradient Boosting Accuracy: {gb_accuracy:.4f}")

del X, y  # Free memory
gc.collect()

✅ Gradient Boosting Accuracy: 0.8201


65

In [ ]:
# ✅ Train XGBoost Model
X_xgb = X_train_pandas
y_xgb = y_train_pandas
xgb_model = xgb.XGBClassifier(n_estimators=50, learning_rate=0.1, tree_method='hist' if GPU_AVAILABLE else 'exact', random_state=42)
xgb_model.fit(X_xgb, y_xgb)
xgb_accuracy = accuracy_score(y_xgb, xgb_model.predict(X_xgb))
print(f"✅ XGBoost Accuracy: {xgb_accuracy:.4f}")

del X_xgb, y_xgb  # Free memory
gc.collect()


✅ XGBoost Accuracy: 0.8201


87

In [ ]:
import tensorflow as tf
print("TF Version:", tf.__version__)
print("Available GPUs:", tf.config.list_physical_devices('GPU'))


TF Version: 2.18.0
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# ✅ Load your dataset (use 'df' as your DataFrame)
data = df.copy()  # Ensure 'df' is loaded before this step

# ✅ Add Technical Indicators (RSI & MACD)
def compute_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))  # Compute RSI
    return rsi  # Return computed RSI values

def compute_macd(data, short=12, long=26, signal=9):
    short_ema = data['Close'].ewm(span=short, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['MACD_Signal'] = data['MACD'].ewm(span=signal, adjust=False).mean()

# ✅ Apply Indicators to `data`
data['RSI'] = compute_rsi(data)  # Ensure RSI is assigned properly
compute_macd(data)  # Compute MACD in place

# ✅ Drop NaN values to avoid errors in training
data.dropna(inplace=True)

# ✅ Verify if 'RSI' column exists
print(data.head())  # Print first few rows to check if RSI is present

# ✅ Drop non-numeric columns before normalization (keep index intact)
data_numeric = data.select_dtypes(include=[np.number])  # Keep only numeric columns

# ✅ Normalize only numeric data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_numeric)

# ✅ Convert back to DataFrame and retain the original index
scaled_df = pd.DataFrame(scaled_data, columns=data_numeric.columns, index=data.index)

# ✅ Check if the data looks correct
print(scaled_df.head())


Price                  Datetime       Close        High         Low  \
32    2022-04-18 17:30:00+00:00  337.783325  338.186676  331.356659   
33    2022-04-18 18:30:00+00:00  335.070007  338.306671  335.070007   
34    2022-04-18 19:30:00+00:00  334.899994  335.111420  332.276672   
35    2022-04-19 13:30:00+00:00  338.873322  341.046661  331.774994   
36    2022-04-19 14:30:00+00:00  342.491669  343.916656  338.646667   

Price        Open   Volume      SMA_20    STD_20  Upper_Band  Lower_Band  \
32     331.663300  2328958  333.171527  4.339015  341.849558  324.493496   
33     337.790009  1617761  333.475694  4.242360  341.960415  324.990973   
34     335.111420  1452827  333.669322  4.213010  342.095343  325.243302   
35     335.250000  5329224  333.839322  4.355812  342.550945  325.127699   
36     338.906647  2771607  334.169905  4.751741  343.673387  324.666423   

Price  Lowest_Low  Highest_High      Stoch  Future_Close  Price_Change  \
32     324.470001    342.079987  75.600994

In [61]:
import gymnasium as gym
import gym_anytrading
import numpy as np
import pandas as pd
import gc
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# ✅ Ensure DataFrame has necessary columns
assert 'Close' in df.columns, "Error: DataFrame must have a 'Close' column"

# ✅ Feature Engineering: Compute Exponential Moving Averages (EMAs)
df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()

# ✅ Create Trading Environment
env = gym.make('stocks-v0', df=df, frame_bound=(10, len(df)), window_size=10)
env = DummyVecEnv([lambda: env])

# ✅ Improved PPO Training with Optimized Hyperparameters
ppo_model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.0001,  # Slightly increased learning rate
    batch_size=512,  # Smaller batch size for better updates
    gamma=0.995,  # High discount factor for trend-following
    n_steps=32768,  # Increased steps for better learning
    ent_coef=0.005,  # Reduce randomness in trading decisions
    vf_coef=0.8,  # Higher value function importance
    clip_range=0.2,  # Prevent large policy changes
)

ppo_model.learn(total_timesteps=500000)  # Train longer for better optimization
ppo_model.save("ppo_trading_model_v4")
gc.collect()

# ✅ Run PPO Trading Strategy with EMA Filtering
obs = env.reset()
done = False
trade_log_rl = []
buy_price = None  # Track if we already hold a position

for i in range(len(df)):
    if i == 0:
        trade_log_rl.append("HOLD")
        continue

    action, _ = ppo_model.predict(obs)

    # EMA-based trend filtering to reduce overtrading
    short_ema = df['EMA_10'].iloc[i]
    long_ema = df['EMA_50'].iloc[i]

    if short_ema > long_ema and buy_price is None:  # Bullish trend
        trade_log_rl.append("BUY")
        buy_price = df['Close'].iloc[i]
    elif short_ema < long_ema and buy_price is not None:  # Bearish trend
        trade_log_rl.append("SELL")
        buy_price = None
    else:
        trade_log_rl.append("HOLD")

df["PPO_Trade_Signal"] = trade_log_rl

# ✅ Implement Enhanced Backtesting with Stop-Loss, Trailing Stop, and Take-Profit
def backtest_rl_trades(price_data, trade_signals, initial_cash=100000,
                        stop_loss_pct=-0.05, take_profit_pct=0.07, trailing_stop_pct=0.04):
    """
    Backtest RL trading strategy with stop-loss, take-profit, and trailing stop.

    Parameters:
    - price_data: DataFrame with stock prices
    - trade_signals: List of 'BUY', 'SELL', or 'HOLD'
    - initial_cash: Starting portfolio balance
    - stop_loss_pct: Stop-loss percentage (-5% default)
    - take_profit_pct: Take-profit percentage (+7% default)
    - trailing_stop_pct: Dynamic trailing stop (-4% from peak)

    Returns:
    - DataFrame with portfolio value, returns, and performance metrics
    """
    capital = initial_cash
    shares = 0
    buy_price = None
    max_price_since_buy = None
    portfolio_values = []

    for i in range(len(trade_signals)):
        price = price_data['Close'].iloc[i]
        action = trade_signals[i]

        if action == "BUY" and capital >= price and buy_price is None:
            shares = capital // price
            buy_price = price
            max_price_since_buy = price  # Track peak price since buy
            capital -= shares * price
            print(f"🟢 BUY at {price}")

        elif action == "SELL" and shares > 0:
            capital += shares * price
            shares = 0
            buy_price = None
            max_price_since_buy = None
            print(f"🔴 SELL at {price}")

        # Trailing stop-loss mechanism
        elif shares > 0:
            max_price_since_buy = max(max_price_since_buy, price)

            if (price - max_price_since_buy) / max_price_since_buy < trailing_stop_pct:
                capital += shares * price
                shares = 0
                buy_price = None
                max_price_since_buy = None
                print(f"⚠️ Trailing stop-loss triggered at {price}")

            elif (price - buy_price) / buy_price < stop_loss_pct:
                capital += shares * price
                shares = 0
                buy_price = None
                max_price_since_buy = None
                print(f"❌ Stop-loss triggered at {price}")

            elif (price - buy_price) / buy_price > take_profit_pct:
                capital += shares * price
                shares = 0
                buy_price = None
                max_price_since_buy = None
                print(f"💰 Take-profit triggered at {price}")

        portfolio_value = capital + (shares * price)
        portfolio_values.append(portfolio_value)

    # Convert to DataFrame
    results = pd.DataFrame({
        'Date': price_data.index,
        'Portfolio Value': portfolio_values
    })

    # Compute returns
    results['Daily Return'] = results['Portfolio Value'].pct_change().fillna(0)
    results['Cumulative Return'] = results['Portfolio Value'] / initial_cash - 1

    # Performance Metrics
    std_dev = results['Daily Return'].std()
    sharpe_ratio = results['Daily Return'].mean() / std_dev * np.sqrt(252) if std_dev != 0 else 0
    max_drawdown = (results['Portfolio Value'].cummax() - results['Portfolio Value']).max() / results['Portfolio Value'].cummax().max()

    # Print Summary
    print(f"📈 Final Portfolio Value: ${results['Portfolio Value'].iloc[-1]:,.2f}")
    print(f"📊 Cumulative Return: {results['Cumulative Return'].iloc[-1]*100:.2f}%")
    print(f"⚡ Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"📉 Max Drawdown: {max_drawdown*100:.2f}%")

    return results

# ✅ Run Backtest
results = backtest_rl_trades(df, trade_log_rl)

# ✅ Compare Performance with Buy & Hold
initial_balance = 100000
shares_held = initial_balance / df['Close'].iloc[0]
final_balance_hold = shares_held * df['Close'].iloc[-1]

# ✅ Compute PPO Trading Performance
balance_ppo = 100000
position = 0

for i, trade in enumerate(trade_log_rl):
    price = df['Close'].iloc[i]

    if trade == "BUY" and position == 0:
        position = balance_ppo / price
        balance_ppo = 0

    elif trade == "SELL" and position > 0:
        balance_ppo = position * price
        position = 0

final_balance_ppo = balance_ppo if balance_ppo > 0 else position * df['Close'].iloc[-1]

# ✅ Compare Results
print(f"📊 PPO Final Balance: ${final_balance_ppo:.2f}")
print(f"📊 Buy & Hold Final Balance: ${final_balance_hold:.2f}")
print("🚀 PPO outperforms!" if final_balance_ppo > final_balance_hold else "📉 Buy & Hold wins!")

gc.collect()


Using cuda device


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


------------------------------
| time/              |       |
|    fps             | 720   |
|    iterations      | 1     |
|    time_elapsed    | 45    |
|    total_timesteps | 32768 |
------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 691           |
|    iterations           | 2             |
|    time_elapsed         | 94            |
|    total_timesteps      | 65536         |
| train/                  |               |
|    approx_kl            | 0.00036232438 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.693        |
|    explained_variance   | -0.000629     |
|    learning_rate        | 0.0001        |
|    loss                 | 41.9          |
|    n_updates            | 10            |
|    policy_gradient_loss | -8.37e-06     |
|    value_loss           | 57.8          |
-----------------------------------

0

In [66]:
# ✅ Compute Buy & Hold Performance
initial_balance = 100000  # Starting balance
shares_held = initial_balance / df['Close'].iloc[0]  # Buy & Hold assumes full investment in stock
buy_hold_final_value = shares_held * df['Close'].iloc[-1]  # Final value after holding

# ✅ Compute Buy & Hold Cumulative Return
buy_hold_cumulative_return = (buy_hold_final_value / initial_balance - 1) * 100
# ✅ Compute Buy & Hold Performance
initial_balance = 100000  # Starting balance
shares_held = initial_balance / df['Close'].iloc[0]  # Buy & Hold assumes full investment in stock
buy_hold_final_value = shares_held * df['Close'].iloc[-1]  # Final value after holding

# ✅ Compute Buy & Hold Cumulative Return
buy_hold_cumulative_return = (buy_hold_final_value / initial_balance - 1) * 100


In [67]:
# ✅ Final Results Summary
print("\n🚀 FINAL RESULTS COMPARISON:")

print("🔹 **Reinforcement Learning (PPO)**")
print(f"📈 PPO Final Portfolio Value: ${final_balance_ppo:,.2f}")
print(f"📊 PPO Cumulative Return: {ppo_cumulative_return:.2f}%")
print(f"⚡ PPO Sharpe Ratio: {ppo_sharpe_ratio:.2f}")
print(f"📉 PPO Max Drawdown: {ppo_max_drawdown:.2f}%")
print(f"📊 PPO Trade Log (First 10): {trade_log_rl[:10]} ...")

print("\n🔹 **Buy & Hold Baseline**")
print(f"📊 Buy & Hold Final Portfolio Value: ${buy_hold_final_value:,.2f}")
print(f"📈 Buy & Hold Cumulative Return: {buy_hold_cumulative_return:.2f}%")

print("\n🔹 **Machine Learning Models (Accuracy Comparison)**")
print(f"📊 Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"📊 Gradient Boosting Accuracy: {gb_accuracy:.4f}")
print(f"📊 XGBoost Accuracy: {xgb_accuracy:.4f}")

# ✅ Declare Winner Based on Portfolio Performance
winner = "PPO" if final_balance_ppo > buy_hold_final_value else "Buy & Hold"
print(f"\n🏆 **Best Strategy Based on Final Portfolio Value: {winner}!**")

gc.collect()



🚀 FINAL RESULTS COMPARISON:
🔹 **Reinforcement Learning (PPO)**
📈 PPO Final Portfolio Value: $284,238.68
📊 PPO Cumulative Return: 184.24%
⚡ PPO Sharpe Ratio: -0.02
📉 PPO Max Drawdown: 8.61%
📊 PPO Trade Log (First 10): ['HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD'] ...

🔹 **Buy & Hold Baseline**
📊 Buy & Hold Final Portfolio Value: $99,915.98
📈 Buy & Hold Cumulative Return: -0.08%

🔹 **Machine Learning Models (Accuracy Comparison)**
📊 Random Forest Accuracy: 0.6992
📊 Gradient Boosting Accuracy: 0.8201
📊 XGBoost Accuracy: 0.8201

🏆 **Best Strategy Based on Final Portfolio Value: PPO!**


40

In [ ]:
import os
import pandas as pd
import joblib
import pickle

# ✅ Save DataFrame
df.to_csv("/content/data_backup.csv", index=False)

# ✅ Save Other Models
joblib.dump(rf_model, "/content/rf_model.pkl")  # Random Forest
joblib.dump(gb_model, "/content/gb_model.pkl")  # Gradient Boosting
joblib.dump(xgb_model, "/content/xgb_model.pkl")  # XGBoost

# ✅ Save PPO Model (Correct Way)
ppo_model.save("/content/ppo_trading_model.zip")  # ✅ Use `.zip` extension

# ✅ Save Scaler (If Used)
with open("/content/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ Data and models saved successfully!")


✅ Data and models saved successfully!

🚀 Now, go to 'Runtime' → 'Restart runtime' to apply changes.

